<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>

# Backpropagation Practice

## *Data Science Unit 4 Sprint 2 Assignment 2*

Implement a 3 input, 4 node hidden-layer, 1 output node Multilayer Perceptron on the following dataset:

| x1 | x2 | x3 | y |
|----|----|----|---|
| 0  | 0  | 1  | 0 |
| 0  | 1  | 1  | 1 |
| 1  | 0  | 1  | 1 |
| 0  | 1  | 0  | 1 |
| 1  | 0  | 0  | 1 |
| 1  | 1  | 1  | 0 |
| 0  | 0  | 0  | 0 |

If you look at the data you'll notice that the first two columns behave like an XOR gate while the last column is mostly just noise. Remember that creating an XOR gate was what the perceptron was criticized for not being able to learn. 

In [1]:
import numpy as np
import pandas as pd

data = { 'x1': [0, 0, 1, 0, 1, 1, 0],
         'x2': [0, 1, 0, 1, 0, 1, 0],
         'x3': [1, 1, 1, 0, 0, 1, 0],
         'y':  [0, 1, 1, 1, 1, 0, 0]
       }

df = pd.DataFrame.from_dict(data).astype('int')

In [7]:
X = df.drop(columns='y').values
y = df[['y']].values

X.shape, y.shape

((7, 3), (7, 1))

In [8]:
# Scale the data
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
X = scaler.fit_transform(X)
X.shape

(7, 3)

In [99]:
# How would I set this up so I can pass different values for inputs, hiddenNodes, and outputNodes
class NeuralNetwork:
    def __init__(self, inputs=3, hiddenNodes=4, outputNodes=1):
        # Set up architcture of neural network
        self.inputs = inputs
        self.hiddenNodes = hiddenNodes
        self.outputNodes = outputNodes
        
        # Initial weights
        # 3x4 matrix array for the first layer
        self.weights1 = np.random.rand(self.inputs, self.hiddenNodes)
        
        # 4x1 matrix array for the second layer
        self.weights2 = np.random.rand(self.hiddenNodes, self.outputNodes)
        
        # Adding bias
        self.Bh = np.ones(hiddenNodes)
        self.Bo = np.ones(outputNodes)
        
    def sigmoid(self, s):
        return 1 / (1 + np.exp(-s))
    
    def sigmoidPrime(self, s):
        # Is this the sigmoid derivative?
        return s * (1 - s)
    
    def feed_forward(self, X):
        """
        Calculate the NN inference using feed forward.
        """
        # Weight sum of inputs
        self.hidden_sum = np.dot(X, self.weights1) + self.Bh
        
        # Activations of weighted sum
        self.activated_hidden = self.sigmoid(self.hidden_sum)
        
        # Weight sum between hidden and output
        self.output_sum = np.dot(self.activated_hidden, self.weights2) + self.Bo
        
        # Final activation of output
        self.activated_output = self.sigmoid(self.output_sum)
        
        return self.activated_output
    
    def backward(self, X, y, o):
        """
        Backward propagate through the network
        """
        
        # Error in Output
        self.o_error = y - o
        
        # Apply derivative of sigmoid to error
        # How far off are we in relation to the Sigmoid f(x) of the output
        self.o_delta = self.o_error * self.sigmoidPrime(o)
        
        # z2 error - not sure what this does
        self.z2_error = self.o_delta.dot(self.weights2.T)
        # How much of the error can be explained by the input => hidden
        self.z2_delta = self.z2_error * self.sigmoidPrime(self.activated_hidden)
        
        # Adjustment to first set of weights (input => hidden)
        self.weights1 += X.T.dot(self.z2_delta)
        # Adjustment to second set of weights (hidden => output)
        self.weights2 += self.activated_hidden.T.dot(self.o_delta)
        # Adjustment to hidden bias
        self.Bh += np.sum(self.z2_delta)
        # Adjustment to output bias
        self.Bo += np.sum(self.o_delta)
        
    def train(self, X, y):
        o = self.feed_forward(X)
        self.backward(X, y, o)

In [20]:
# Train the 'net
nn = NeuralNetwork()

# Number of Epochs / Iterations
for i in range(10000):
    if (i+1 in [1, 2, 3, 4, 5]) or ((i+1) % 1000 == 0):
        print('+' + '---' * 3 + f'EPOCH {i+1}' * 3 + '+')
        print('Input: \n', X)
        print('Actual Output: \n', y)
        print('Predicted Output: \n', str(nn.feed_forward(X)))
        print('Loss: \n', str(np.mean(np.square(y - nn.feed_forward(X)))))
    nn.train(X, y)

+---------EPOCH 1EPOCH 1EPOCH 1+
Input: 
 [[0. 0. 1.]
 [0. 1. 1.]
 [1. 0. 1.]
 [0. 1. 0.]
 [1. 0. 0.]
 [1. 1. 1.]
 [0. 0. 0.]]
Actual Output: 
 [[0]
 [1]
 [1]
 [1]
 [1]
 [0]
 [0]]
Predicted Output: 
 [[0.76320133]
 [0.8110523 ]
 [0.80842472]
 [0.78161842]
 [0.77847231]
 [0.83839147]
 [0.72061243]]
Loss: 
 0.2819751655207015
+---------EPOCH 2EPOCH 2EPOCH 2+
Input: 
 [[0. 0. 1.]
 [0. 1. 1.]
 [1. 0. 1.]
 [0. 1. 0.]
 [1. 0. 0.]
 [1. 1. 1.]
 [0. 0. 0.]]
Actual Output: 
 [[0]
 [1]
 [1]
 [1]
 [1]
 [0]
 [0]]
Predicted Output: 
 [[0.68347587]
 [0.72277802]
 [0.72062503]
 [0.69869403]
 [0.69631221]
 [0.74743194]
 [0.65180711]]
Loss: 
 0.25550860529144126
+---------EPOCH 3EPOCH 3EPOCH 3+
Input: 
 [[0. 0. 1.]
 [0. 1. 1.]
 [1. 0. 1.]
 [0. 1. 0.]
 [1. 0. 0.]
 [1. 1. 1.]
 [0. 0. 0.]]
Actual Output: 
 [[0]
 [1]
 [1]
 [1]
 [1]
 [0]
 [0]]
Predicted Output: 
 [[0.61868589]
 [0.64613499]
 [0.64467197]
 [0.62936438]
 [0.62783288]
 [0.66437905]
 [0.59772043]]
Loss: 
 0.24411415058755465
+---------EPOCH 4EPO

## Try building/training a more complex MLP on a bigger dataset.

Use the [MNIST dataset](http://yann.lecun.com/exdb/mnist/) to build the cannonical handwriting digit recognizer and see what kind of accuracy you can achieve. 

If you need inspiration, the internet is chalk-full of tutorials, but I want you to see how far you can get on your own first. I've linked to the original MNIST dataset above but it will probably be easier to download data through a neural network library. If you reference outside resources make sure you understand every line of code that you're using from other sources, and share with your fellow students helpful resources that you find.


### Parts
1. Gathering & Transforming the Data
2. Making MNIST a Binary Problem
3. Estimating your Neural Network (the part you focus on)

### Gathering the Data 

`keras` has a handy method to pull the mnist dataset for you. You'll notice that each observation is a 28x28 arrary which represents an image. Although most Neural Network frameworks can handle higher dimensional data, that is more overhead than necessary for us. We need to flatten the image to one long row which will be 784 values (28X28). Basically, you will be appending each row to one another to make on really long row. 

In [100]:
import numpy as np
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical

In [101]:
# input image dimensions
img_rows, img_cols = 28, 28

In [102]:
# the data, split between train and test sets
(x_train, y_train), (x_test, y_test) = mnist.load_data()

In [103]:
x_train = x_train.reshape(x_train.shape[0], img_rows * img_cols)
x_test = x_test.reshape(x_test.shape[0], img_rows * img_cols)

# Normalize Our Data
x_train = x_train / 255
x_test = x_test / 255

In [104]:
# Now the data should be in a format you're more familiar with
x_train.shape

(60000, 784)

### Making MNIST a Binary Problem 
MNIST is multiclass classification problem; however we haven't covered all the necessary techniques to handle this yet. You would need to one-hot encode the target, use a different loss metric, and use softmax activations for the last layer. This is all stuff we'll cover later this week, but let us simply the problem for now: Zero or all else.

In [105]:
import numpy as np

y_temp = np.zeros(y_train.shape)
y_temp[np.where(y_train == 0.0)[0]] = 1
y_train = y_temp

y_temp = np.zeros(y_test.shape)
y_temp[np.where(y_test == 0.0)[0]] = 1
y_test = y_temp

In [106]:
# A Nice Binary target for ya to work with
y_train.shape

(60000,)

In [107]:
# Combine X and y columns to take a sample
train = np.hstack((x_train, y_train.reshape(-1, 1)))
test = np.hstack((x_test, y_test.reshape(-1, 1)))

train.shape, test.shape

((60000, 785), (10000, 785))

In [108]:
train = pd.DataFrame(train)
test = pd.DataFrame(test)
train.shape, test.shape

((60000, 785), (10000, 785))

In [109]:
train = train.sample(frac=0.25, axis=0)
test = test.sample(frac=0.25, axis=0)
train.shape, test.shape

((15000, 785), (2500, 785))

In [110]:
X_train = train.drop(columns=784).to_numpy()
y_train = train[784].to_numpy()
X_test = test.drop(columns=784).to_numpy()
y_test = test[784].to_numpy()

X_train.shape, y_train.shape, X_test.shape, y_test.shape

((15000, 784), (15000,), (2500, 784), (2500,))

In [111]:
y_train = y_train.reshape(-1, 1)
y_test = y_test.reshape(-1, 1)
X_train.shape, y_train.shape, X_test.shape, y_test.shape

((15000, 784), (15000, 1), (2500, 784), (2500, 1))

### Estimating Your `net

In [113]:
# Train the 'net
nn = NeuralNetwork(inputs=784, hiddenNodes=50, outputNodes=1)

# Number of Epochs / Iterations
for i in range(1000):
    if (i+1 in [1, 2, 3, 4, 5]) or ((i+1) % 1000 == 0):
        print('+' + '---' * 3 + f'EPOCH {i+1}' * 3 + '+')
        print('Input: \n', X_test)
        print('Actual Output: \n', y_test)
        print('Predicted Output: \n', str(nn.feed_forward(X_test)))
        print('Loss: \n', str(np.mean(np.square(y_test - nn.feed_forward(X_test)))))
    nn.train(X_train, y_train)

+---------EPOCH 1EPOCH 1EPOCH 1+
Input: 
 [[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
Actual Output: 
 [[0.]
 [0.]
 [0.]
 ...
 [0.]
 [0.]
 [0.]]
Predicted Output: 
 [[1.]
 [1.]
 [1.]
 ...
 [1.]
 [1.]
 [1.]]
Loss: 
 0.9083999999882512
+---------EPOCH 2EPOCH 2EPOCH 2+
Input: 
 [[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
Actual Output: 
 [[0.]
 [0.]
 [0.]
 ...
 [0.]
 [0.]
 [0.]]
Predicted Output: 
 [[1.]
 [1.]
 [1.]
 ...
 [1.]
 [1.]
 [1.]]
Loss: 
 0.9083999999882512
+---------EPOCH 3EPOCH 3EPOCH 3+
Input: 
 [[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
Actual Output: 
 [[0.]
 [0.]
 [0.]
 ...
 [0.]
 [0.]
 [0.]]
Predicted Output: 
 [[1.]
 [1.]
 [1.]
 ...
 [1.]
 [

## Stretch Goals: 

- Make MNIST a multiclass problem using cross entropy & soft-max
- Implement Cross Validation model evaluation on your MNIST implementation 
- Research different [Gradient Descent Based Optimizers](https://keras.io/optimizers/)
 - [Siraj Raval the evolution of gradient descent](https://www.youtube.com/watch?v=nhqo0u1a6fw)
- Build a housing price estimation model using a neural network. How does its accuracy compare with the regression models that we fit earlier on in class?